# Eat Safe, Love

## Notebook Set Up

In [5]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd
from decimal import Decimal

In [6]:
# Create an instance of MongoClient
client = MongoClient()

In [7]:
# assign the uk_food database to a variable name
db = client.uk_food

In [8]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [9]:
# assign the collection to a variable
collection = db.establishments

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [10]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene':20}
# Use count_documents to display the number of documents in the result
count = collection.count_documents(query)
print(f"Number of establishments with a hygiene score of 20: {count}")
# Display the first document in the results using pprint
document = collection.find_one(query)
print("First document in results:")
pprint(document)

Number of establishments with a hygiene score of 20: 41
First document in results:
{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': '0',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('642b0ee9f2cee067fa0047c9'),
 'geocode': {'latitude': Decimal128('50.769705'),
             'longitude': Decimal128('0.27694')},
 'links': [{'href': 'https://api

In [11]:
# Convert the result to a Pandas DataFrame

# Display the number of rows in the DataFrame

# Display the first 10 rows of the DataFrame
df = pd.DataFrame(list(collection.find({"scores.Hygiene": "20"})))
print(f"\nNumber of rows in DataFrame: {len(df)}")
print("First 10 rows:")
print(df.head(10))


Number of rows in DataFrame: 0
First 10 rows:
Empty DataFrame
Columns: []
Index: []


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [16]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = query = {'LocalAuthorityName':{'$regex':"London"}, 'RatingValue' :{'$gte':"4"}}


# Use count_documents to display the number of documents in the result
print(f"Number of establishments with London as the Local Authority and a RatingValue greater than or equal to 4: {count}")

# Display the first document in the results using pprint
document = collection.find_one(query)
print("First document in results:")
pprint(document)

Number of establishments with London as the Local Authority and a RatingValue greater than or equal to 4: 41
First document in results:
{'AddressLine1': 'Oak Apple Farm Building 103 Sheernes Docks',
 'AddressLine2': 'Sheppy Kent',
 'AddressLine3': '',
 'AddressLine4': '',
 'BusinessName': "Charlie's",
 'BusinessType': 'Other catering premises',
 'BusinessTypeID': 7841,
 'ChangesByServerID': 0,
 'Distance': 4627.439467780196,
 'FHRSID': 621707,
 'LocalAuthorityBusinessID': 'PI/000025307',
 'LocalAuthorityCode': '508',
 'LocalAuthorityEmailAddress': 'publicprotection@cityoflondon.gov.uk',
 'LocalAuthorityName': 'City of London Corporation',
 'LocalAuthorityWebSite': 'http://www.cityoflondon.gov.uk/Corporation/homepage.htm',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'ME12',
 'RatingDate': '2021-10-18T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': '4',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('642b0eeaf2cee067fa0061e8'),
 'geocode': {'latitude': D

In [19]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(list(collection.find(query)))

# Display the number of rows in the DataFrame

# Display the first 10 rows of the DataFrame
print(f"\nNumber of rows in DataFrame: {len(df)}")
print("First 10 rows:")
print(df.head(10))


Number of rows in DataFrame: 34
First 10 rows:
                        _id   FHRSID  ChangesByServerID  \
0  642b0eeaf2cee067fa0061e8   621707                  0   
1  642b0eeaf2cee067fa006512  1130836                  0   
2  642b0eeaf2cee067fa00705b   293783                  0   
3  642b0eeaf2cee067fa007e5e  1315095                  0   
4  642b0eeaf2cee067fa007e61   294474                  0   
5  642b0eeaf2cee067fa008568   294900                  0   
6  642b0eebf2cee067fa00a672  1252616                  0   
7  642b0eebf2cee067fa00a676   293756                  0   
8  642b0eebf2cee067fa00aa39   878523                  0   
9  642b0eebf2cee067fa00aa55   293772                  0   

  LocalAuthorityBusinessID                          BusinessName  \
0             PI/000025307                             Charlie's   
1             PI/000034075               Mv City Cruises Erasmus   
2             PI/000002614             Benfleet Motor Yacht Club   
3             PI/000036464  Co

### 3. What are the top 5 establishments with a `RatingValue` rating value of '5', sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [22]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.490142
longitude = 0.08384

query = {
    "geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search},
    "RatingValue": "5"
}
sort = [("scores.Hygiene", 1)]

# Print the results
result = collection.find(query).sort(sort).limit(5)

In [23]:
# Convert result to Pandas DataFrame
df = pd.DataFrame(list(result))
print(f"\nNumber of rows in DataFrame: {len(df)}")
print("First 10 rows:")
print(df.head(10))


Number of rows in DataFrame: 5
First 10 rows:
                        _id   FHRSID  ChangesByServerID  \
0  642b0eebf2cee067fa00a031   694609                  0   
1  642b0eebf2cee067fa00a048   695241                  0   
2  642b0eebf2cee067fa00a04b   694478                  0   
3  642b0eebf2cee067fa00a004   695223                  0   
4  642b0eebf2cee067fa00a012  1380578                  0   

  LocalAuthorityBusinessID                         BusinessName  \
0             PI/000116619                            Volunteer   
1             PI/000179088              Plumstead Manor Nursery   
2             PI/000086506                    Atlantic Fish Bar   
3             PI/000178842                              Iceland   
4                    14425  Howe and Co Fish and Chips - Van 17   

                            BusinessType  BusinessTypeID  \
0                      Pub/bar/nightclub            7843   
1                        Caring Premises               5   
2              

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [27]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
pipeline = [
    {"$match": {"scores.Hygiene": "0"}},
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}}
]

result = list(collection.aggregate(pipeline))



# Print the number of documents in the result
print(f"Number of establishments with a hygiene score of 0, grouped by Local Authority and sorted from highest to lowest: {len(result)}")



# Print the first 10 results
print("First 10 results:")
pprint(result[:10])



Number of establishments with a hygiene score of 0, grouped by Local Authority and sorted from highest to lowest: 0
First 10 results:
[]


In [30]:
# Convert the result to a Pandas DataFrame

df = pd.DataFrame(result)

# Display the number of rows in the DataFrame

print(f"Number of rows in DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame

print("First 10 rows:")
print(df.head(10))

Number of rows in DataFrame: 0
First 10 rows:
Empty DataFrame
Columns: []
Index: []
